<a href="https://colab.research.google.com/github/LuigiAjello/PlanoEmergencialConectividadeNacionalOtimizacao/blob/main/Plano_Emergencial_Conectividade_Nacional_Otimizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto: Plano Emergencial de Conectividade Nacional em Situação de Crise Aérea**


---


### I  Contexto
Imagine um cenário extremo: Um **surto pandêmico** causa suspensão repentina da maior parte da malha aérea brasileira e apenas **um avião governamental** permanece disponível para transportar suprimentos ou equipes médicas.
O desafio do Ministério da Infra-estrutura é **partir de Brasília, visitar as 27 capitais e retornar, no menor tempo total**, usando somente rotas diretas realmente voadas em `Dezembro/ 2024` **(dados VRA – ANAC)**.

[**LINK-DATASET**](https://siros.anac.gov.br/siros/registros/diversos/vra/)

### II  Objetivo técnico

1.   Minimizar o tempo total de voo (soma dos tempos médios por rota).
2.   Partir e terminar em SBBR (Brasília).
3.   Garantir que todas as capitais sejam atendidas — repetição de escalas permitida se reduzir custo.
4.   Operar apenas voos diretos existentes em Dezembro/2024.

### III  Dados
Fonte: Voos Realizados ANAC (VRA_2024_12.xlsx).

Inicialmente foi utilizado o mês de julho de 2024 como base de dados; no entanto, constatou-se a ausência de voos diretos para o aeroporto de Porto Alegre nesse período, o que inviabilizou sua utilização :





> Em julho de 2024, o Aeroporto de Porto Alegre (SBBPA) ainda estava em processo de restabelecimento após as grandes enchentes de maio 2024, que inundaram a pista principal com até 2,5 m de água.

>A ANAC chegou a suspender temporariamente todos os voos no terminal. Mesmo com a retomada parcial de operações em 15 de julho, o aeroporto funcionava com capacidade limitada e passageiros foram atendidos por uma instalação improvisada em Canoas

>Consequentemente, não houve voos diretos no VRA de julho de 2024 para Porto Alegre, o que tornou o uso desse mês impossível para compor um itinerário completo. Por isso, você não encontrou rotas envolvendo essa capital nesse período.



**Tratamento**


*   **Filtragem de colunas:** mantidas no final apenas *Origem*, *Destino*, *duração*
*   Conversão de Partida/Chegada para datetime
*   Cálculo da duração em minutos para cada voo
*   Filtro Situação Voo = REALIZADO
* Média de minutos por par Origem → Destino
* Retenção somente dos 27 ICAO das capitais
*Remoção de loops (Origem = Destino) e de duplicatas




### **1. Fonte / Import bibliotecas iniciais :**


---



Aqui importamos a biblioteca inicial e a fonte

In [226]:
import pandas
from IPython.display import display
df = pandas.read_excel('/content/VRA_2024_12_excel.xlsx')
display(df)

,Empresa Aérea,Número Voo,Sigla ICAO Aeroporto Origem,Descrição Aeroporto Origem,Partida Real,Sigla ICAO Aeroporto Destino,Descrição Aeroporto Destino,Chegada Real,Situação Voo,Situação Partida,Situação Chegada
0,"AMERICAN AIRLINES, INC.",904,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,01/12/2024 23:51,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",02/12/2024 08:16,REALIZADO,Antecipado,Atraso 30-60
1,"AMERICAN AIRLINES, INC.",905,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",01/12/2024 00:55,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,01/12/2024 09:07,REALIZADO,Antecipado,Antecipado
2,"AMERICAN AIRLINES, INC.",906,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,01/12/2024 00:27,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",01/12/2024 08:42,REALIZADO,Antecipado,Antecipado
3,"AMERICAN AIRLINES, INC.",925,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",01/12/2024 23:54,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,02/12/2024 08:12,REALIZADO,Pontual,Pontual
4,"AMERICAN AIRLINES, INC.",929,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",01/12/2024 22:30,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,02/12/2024 06:30,REALIZADO,Atraso 60-120,Pontual
...,...,...,...,...,...,...,...,...,...,...,...
86009,CONSORCIO VENEZOLANO DE INDUSTRIAS AERONAUTICA...,4931,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,12/12/2024 17:29,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,12/12/2024 19:40,REALIZADO,NaN,NaN
86010,CONSORCIO VENEZOLANO DE INDUSTRIAS AERONAUTICA...,4930,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,19/12/2024 13:44,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,19/12/2024 15:53,REALIZADO,Pontual,Pontual
86011,CONSORCIO VENEZOLANO DE INDUSTRIAS AERONAUTICA...,4931,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,19/12/2024 17:04,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,19/12/2024 19:15,REALIZADO,Pontual,Pontual
86012,CONSORCIO VENEZOLANO DE INDUSTRIAS AERONAUTICA...,4930,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,26/12/2024 14:07,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,26/12/2024 16:19,REALIZADO,Atraso 30-60,Atraso 30-60


### **2. Pré-processamento dos Dados**

---
Nesta etapa, os dados brutos são limpos, transformados e validados para que possam ser utilizados corretamente no modelo de otimização.


---



### **2.1 - Filtragem de colunas restantes**
Foram removidas do DataFrame as colunas Empresa Aérea, Número Voo, Situação Partida e Situação Chegada, mantendo apenas os campos estritamente necessários para o cálculo das rotas e tempos médios


In [227]:
df = df.drop(columns=['Empresa Aérea',"Número Voo","Situação Partida"	,"Situação Chegada"])
display(df)

,Sigla ICAO Aeroporto Origem,Descrição Aeroporto Origem,Partida Real,Sigla ICAO Aeroporto Destino,Descrição Aeroporto Destino,Chegada Real,Situação Voo
0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,01/12/2024 23:51,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",02/12/2024 08:16,REALIZADO
1,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",01/12/2024 00:55,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,01/12/2024 09:07,REALIZADO
2,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,01/12/2024 00:27,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",01/12/2024 08:42,REALIZADO
3,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",01/12/2024 23:54,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,02/12/2024 08:12,REALIZADO
4,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",01/12/2024 22:30,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,02/12/2024 06:30,REALIZADO
...,...,...,...,...,...,...,...
86009,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,12/12/2024 17:29,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,12/12/2024 19:40,REALIZADO
86010,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,19/12/2024 13:44,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,19/12/2024 15:53,REALIZADO
86011,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,19/12/2024 17:04,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,19/12/2024 19:15,REALIZADO
86012,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,26/12/2024 14:07,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,26/12/2024 16:19,REALIZADO


### **2.2- Cálculo da duração dos voos**
1- Convertidas as colunas Partida Real e Chegada Real para formato datetime.

2-  Calculada a diferença Chegada – Partida (Duracao).

3- Transformada a duração em minutos (Duracao Minutos), valor que servirá como peso das arestas no grafo de otimização.

In [228]:
import pandas as pd

# Converter colunas para datetime
df['Partida Real'] = pd.to_datetime(df['Partida Real'], format="%d/%m/%Y %H:%M")
df['Chegada Real'] = pd.to_datetime(df['Chegada Real'], format="%d/%m/%Y %H:%M")

# Calcular a duração
df['Duracao'] = df['Chegada Real'] - df['Partida Real']

# Calcular duração total em minutos (peso para o grafo)
df['Duracao Minutos'] = df['Duracao'].dt.total_seconds() / 60

# Visualizar resultado
display(df)


,Sigla ICAO Aeroporto Origem,Descrição Aeroporto Origem,Partida Real,Sigla ICAO Aeroporto Destino,Descrição Aeroporto Destino,Chegada Real,Situação Voo,Duracao,Duracao Minutos
0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,2024-12-01 23:51:00,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",2024-12-02 08:16:00,REALIZADO,0 days 08:25:00,505.0
1,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",2024-12-01 00:55:00,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,2024-12-01 09:07:00,REALIZADO,0 days 08:12:00,492.0
2,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,2024-12-01 00:27:00,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",2024-12-01 08:42:00,REALIZADO,0 days 08:15:00,495.0
3,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",2024-12-01 23:54:00,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,2024-12-02 08:12:00,REALIZADO,0 days 08:18:00,498.0
4,KMIA,"MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA -...",2024-12-01 22:30:00,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,2024-12-02 06:30:00,REALIZADO,0 days 08:00:00,480.0
...,...,...,...,...,...,...,...,...,...
86009,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,2024-12-12 17:29:00,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,2024-12-12 19:40:00,REALIZADO,0 days 02:11:00,131.0
86010,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,2024-12-19 13:44:00,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,2024-12-19 15:53:00,REALIZADO,0 days 02:09:00,129.0
86011,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,2024-12-19 17:04:00,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,2024-12-19 19:15:00,REALIZADO,0 days 02:11:00,131.0
86012,SVPR,MANUEL CARLOS PIAR GUAYANA AIRPORT - CIUDAD GU...,2024-12-26 14:07:00,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,2024-12-26 16:19:00,REALIZADO,0 days 02:12:00,132.0


### **2.3- Agregação das rotas**
1 - Filtrados apenas registros com Situação Voo = “REALIZADO” e minuto válido.

2 - Agrupamento por Origem ICAO × Destino ICAO.

3 - Calculado o tempo médio de voo (tempo_medio_minutos) para cada par.

4 - Resultado: um dataset enxuto contendo uma linha por rota direta com o respectivo tempo médio — pronto para ser filtrado pelas capitais.

In [229]:
rotas = (
    df[
        (df["Situação Voo"] == "REALIZADO") & # Filtrar por voos "REALIZADO"
        (df["Duracao Minutos"].notna())      # Garantir que 'Duracao Minutos' não seja NaN
    ]
    .groupby(["Sigla ICAO Aeroporto Origem", "Sigla ICAO Aeroporto Destino"])
    .agg(tempo_medio_minutos=("Duracao Minutos", "mean"))
    .reset_index()
)


display(rotas)

,Sigla ICAO Aeroporto Origem,Sigla ICAO Aeroporto Destino,tempo_medio_minutos
0,CYUL,SBGR,595.366667
1,CYYZ,SBGR,594.451613
2,DAAG,GQNO,330.000000
3,DGAA,LCLK,390.000000
4,DGAA,SBGR,439.428571
...,...,...,...
1579,TTPP,SBGL,521.000000
1580,UBBB,EBLG,305.000000
1581,VTBS,OMSJ,390.000000
1582,ZBAA,LEMD,707.000000


### **2.4- Filtro das capitais**
1 - Definido o conjunto dos 27 ICAO que representam as capitais brasileiras.

2 - Mantidas no dataset apenas as rotas cujo origem e destino pertencem a esse conjunto.

3 - Resultado: df_capitais — malha de rotas diretas exclusivamente entre capitais, pronta para verificação de integridade e uso no modelo de otimização.

In [230]:
capitais_icao = {
    "Rio Branco": "SBRB",
    "Maceió": "SBMO",
    "Macapá": "SBMQ",
    "Manaus": "SBEG",
    "Salvador": "SBSV",
    "Fortaleza": "SBFZ",
    "Brasília": "SBBR",
    "Vitória": "SBVT",
    "Goiânia": "SBGO",
    "São Luís": "SBSL",
    "Cuiabá": "SBCY",
    "Campo Grande": "SBCG",
    "Belo Horizonte": "SBCF",
    "Belém": "SBBE",
    "João Pessoa": "SBJP",
    "Curitiba": "SBCT",
    "Recife": "SBRF",
    "Teresina": "SBTE",
    "Rio de Janeiro": "SBGL",
    "Natal": "SBSG",
    "Porto Alegre": "SBPA",
    "Porto Velho": "SBPV",
    "Boa Vista": "SBBV",
    "Florianópolis": "SBFL",
    "São Paulo": "SBGR",
    "Aracaju": "SBAR",
    "Palmas": "SBPJ"
}


capitais_set = set(capitais_icao.values())


df_capitais = rotas[
    rotas["Sigla ICAO Aeroporto Origem"].isin(capitais_set) &
    rotas["Sigla ICAO Aeroporto Destino"].isin(capitais_set)
].copy()

print(f"Rotas diretas entre capitais: {len(df_capitais)}")

Rotas diretas entre capitais: 328


### **2.5-  Verificação de integridade**

1 - Contados os ICAO únicos presentes em Origem ∪ Destino.

2 - Exibido o total e a lista (para auditoria).

3 - Assert garante que o conjunto contenha exatamente 27 capitais – se não, interrompe o fluxo.

In [231]:
# Colunas padrão usadas
col_origem  = "Sigla ICAO Aeroporto Origem"
col_destino = "Sigla ICAO Aeroporto Destino"

# 1) conjunto de ICAO distintos nas duas colunas
icos_unicos = set(df_capitais[col_origem]) | set(df_capitais[col_destino])

# 2) contagem
qtd_aeroportos = len(icos_unicos)
print(f"Número de aeroportos únicos no dataset: {qtd_aeroportos}")

# 3) listar se quiser conferir
print(sorted(icos_unicos))

# 4) checagem de segurança
assert qtd_aeroportos == 27, "O dataset NÃO tem exatamente 27 aeroportos-capitais!"

Número de aeroportos únicos no dataset: 27
['SBAR', 'SBBE', 'SBBR', 'SBBV', 'SBCF', 'SBCG', 'SBCT', 'SBCY', 'SBEG', 'SBFL', 'SBFZ', 'SBGL', 'SBGO', 'SBGR', 'SBJP', 'SBMO', 'SBMQ', 'SBPA', 'SBPJ', 'SBPV', 'SBRB', 'SBRF', 'SBSG', 'SBSL', 'SBSV', 'SBTE', 'SBVT']


## **2.6-  Remoção de loops**

1 - Identificadas rotas com Origem = Destino (loops operacionais).

2 - Encontradas len(df_cap_loops) ocorrências; todas foram excluídas.

3 - df_capitais atualizado fica livre de auto-arestas, pronto para a etapa de otimização.

In [232]:

# 1) Todas as rotas em que origem == destino
df_cap_loops = df_capitais[
    df_capitais["Sigla ICAO Aeroporto Origem"] ==
    df_capitais["Sigla ICAO Aeroporto Destino"]
].copy()

print(f"{len(df_cap_loops)} linhas com origem = destino no df_capitais")

df_capitais = df_capitais.drop(df_cap_loops.index).copy()
display(df_capitais)

21 linhas com origem = destino no df_capitais


,Sigla ICAO Aeroporto Origem,Sigla ICAO Aeroporto Destino,tempo_medio_minutos
203,SBAR,SBBR,127.727273
204,SBAR,SBCF,121.416667
205,SBAR,SBGL,149.166667
206,SBAR,SBGR,163.116667
208,SBAR,SBRF,76.142857
...,...,...,...
1413,SBVT,SBCF,59.076271
1414,SBVT,SBGL,67.557078
1415,SBVT,SBGR,94.954717
1419,SBVT,SBRF,129.800000


### **3. Otimizacao em rede**

---
Nesta etapa,transformamos o conjunto de rotas em um grafo, formulamos um modelo inteiro para minimizar o tempo total de voo (partindo e retornando a Brasília) e resolvemos o problema com PuLP


---



```
#Instalar dependências e importar
```



In [233]:
#!pip install pulp
import pulp

## **3.1-  Configuração do modelo**

1 - Extrai arestas e seus tempos médios (arcs).

2 - Define o conjunto de nós (nodes) e verifica as 27 capitais.


3 - Fixa SBBR como ponto de partida/retorno e calcula N = 26 para o fluxo.




In [234]:
# ------------------------------------------------------------
# 3.1 | Configuração inicial do modelo
# ------------------------------------------------------------

# --- Conjunto de arestas (origem, destino, tempo médio) -----
arcs = list(
    df_capitais[["Sigla ICAO Aeroporto Origem",
                 "Sigla ICAO Aeroporto Destino",
                 "tempo_medio_minutos"]]
    .itertuples(index=False, name=None)
)

# --- Nó de cada capital (27 ICAO) ---------------------------
nodes = sorted(
    set(df_capitais["Sigla ICAO Aeroporto Origem"])
  | set(df_capitais["Sigla ICAO Aeroporto Destino"])
)
assert len(nodes) == 27, "Capital faltando!"

# --- Ponto de partida / retorno -----------------------------
start = "SBBR"   # Brasília

# --- Quantidade de capitais a “consumir” no fluxo -----------
N = len(nodes) - 1   # = 26

## **3.2-  Definição das variáveis de decisão**

1 - Cria o modelo linear inteiro (LpProblem).


2 - xᵢⱼ (0 ou 1) – “vou pegar esse voo direto?” (1 = sim, 0 = não).


3 - fᵢⱼ (0 a 26) – contador que sai de Brasília e vai caindo 1 em cada capital; faz o caminho ficar todo ligado e voltar ao início.


In [235]:
# ------------------------------------------------------------
# 3.2 | Definição das variáveis de decisão
# ------------------------------------------------------------
# Modelo de Programação Linear   (queremos minimizar o tempo total de voo)
m = pulp.LpProblem("Rota_27_Capitais_Com_Escalas", pulp.LpMinimize)

# x[(i, j, t)]  -> 1 se o voo direto i→j for escolhido, 0 caso contrário
# OBS.: em `LpVariable.dicts`, o conjunto de índices (arcs) deve ser passado
#       como argumento POSICIONAL, não como keyword (por isso o erro).
x = pulp.LpVariable.dicts(
        "x",            # nome-base das variáveis
        arcs,           # <- índices POSICIONAIS (origem, destino, tempo)
        lowBound=0,
        upBound=1,
        cat="Binary"
)

# f[(i, j, t)]  -> quantidade de “unidades” que percorre a rota
#                 (0 a 26) para assegurar que todo o caminho fique conectado
f = pulp.LpVariable.dicts(
        "f",
        arcs,           # idem: índices posicional
        lowBound=0,
        upBound=N,      # N = 26 (uma unidade para cada capital fora de Brasília)
        cat="Continuous"
)

## **3.3-  Definição da função-objetivo**
1 - Pegue todos os voos que marcamos como “usar” e some seus tempos.

2 - Minimiza o tempo total percorrido


In [236]:
# ------------------------------------------------------------
# 3.3 | Objetivo: deixar o tempo total o menor possível
# ------------------------------------------------------------
# Para cada voo (i → j) com tempo t:
#   - se x = 1 (voo entra na rota), conta t minutos
#   - se x = 0 (voo fora da rota), conta 0
# A soma final é o tempo total, que o solver tenta minimizar.
m += pulp.lpSum(x[(i, j, t)] * t for (i, j, t) in arcs)

## **3.4- Regras que o caminho deve obedecer**
1 - **Entra o mesmo que sai:**


> Para cada capital, o número de voos que chegam tem de ser igual ao número de voos que saem. Assim o avião não “aparece” nem “desaparece” do nada.


2 - **Visita obrigatória:**


> Se a cidade não é Brasília, pelo menos um voo tem de sair dela — isso garante que o avião passe por lá.

*(Brasília,  já é ponto de partida / chegada)*

3 - **Fluxo válido:**

> O contador f anda somente em rotas ativas (x = 1) → sem caminhos f



In [237]:
# ------------------------------------------------------------
# 3.4 | Restrições de grau e de fluxo (explicadas acima)
# ------------------------------------------------------------
for n in nodes:
    out_edges = [(i, j, t) for (i, j, t) in arcs if i == n]   # voos que saem do nó n
    in_edges  = [(i, j, t) for (i, j, t) in arcs if j == n]   # voos que chegam ao nó n

    # 1. entra = sai
    m += pulp.lpSum(x[e] for e in out_edges) == pulp.lpSum(x[e] for e in in_edges)

    # 2. toda capital fora de Brasília precisa aparecer
    if n != start:
        m += pulp.lpSum(x[e] for e in out_edges) >= 1

    # 3. fluxo pode passar só se a aresta existir (x = 1)
    for e in out_edges:
        m += f[e] <= N * x[e]     # N = 26 (limite máximo do fluxo)


## **3.5-   Garantia de percurso completo**

1 - **Brasília** emite 26 unidades de contagem — uma para cada capital remanescente.


2 - **Cada capital** absorve 1 unidade quando é visitada, registrando sua passagem.


3 - Ao final, todas as 26 unidades ficam distribuídas nas capitais e o caminho retorna a Brasília, comprovando que o trajeto é único e **cobre o país inteiro**.







In [238]:
# ------------------------------------------------------------
# 3.5 | Regras de contagem – conectividade do percurso
# ------------------------------------------------------------

# Brasília emite 26 unidades
m += (
    pulp.lpSum(f[e] for e in arcs if e[0] == start)   # unidades que saem de SBBR
  - pulp.lpSum(f[e] for e in arcs if e[1] == start)   # unidades que retornam a SBBR
  == N                                                # N = 26 unidades distribuídas
)

# Cada capital diferente de Brasília recebe exatamente 1 unidade
for n in nodes:
    if n == start:
        continue
    m += (
        pulp.lpSum(f[e] for e in arcs if e[1] == n)   # unidades que chegam a n
      - pulp.lpSum(f[e] for e in arcs if e[0] == n)   # unidades que saem de n
      == 1
    )


## **3.6 – Resolver, pegar a rota e mostrar o resultado**
1 - Executa o solver CBC (limite 10 min) e encontra a rota de menor tempo.

2 - Extrai as arestas escolhidas (tour_edges) e mostra quantos voos foram usados.

3 - Constrói o grafo solução e ordena o percurso com nx.eulerian_circuit, começando em Brasília.

4 - Confirma que as 27 capitais estão presentes.

5 - Exibe o status do modelo, o tempo total em minutos e a sequência final de capitais visitadas.

In [239]:
# ------------------------------------------------------------
# 3.6 | Resolver, extrair solução e exibir
# ------------------------------------------------------------

# 1) Rodar o solver (até 600 s)
solver = pulp.PULP_CBC_CMD(msg=True, timeLimit=600)
m.solve(solver)                      # calcula a rota ótima

# 2) Arestas selecionadas (x = 1)
tour_edges = [
    (i, j) for (i, j, t) in arcs
    if pulp.value(x[(i, j, t)]) > 0.5
]
print("Número de conexões (voos):", len(tour_edges))

# 3) Reconstruir a ordem da viagem
Gsol = nx.DiGraph()
Gsol.add_edges_from(tour_edges)

euler_circuit = list(nx.eulerian_circuit(Gsol, source=start))
ordered = [start] + [v for (_, v) in euler_circuit]

# 4) Checagem – todas as 27 capitais apareceram?
missing = set(nodes) - set(ordered)
assert not missing, f"Capitais não listadas: {missing}"

# 5) Mostrar resultado
print("Status:", pulp.LpStatus[m.status])
print("Custo total (min):", round(pulp.value(m.objective), 2))
print("Sequência de visita:")
print(" → ".join(ordered))

Número de conexões (voos): 28
Status: Optimal
Custo total (min): 2121.46
Sequência de visita:
SBBR → SBPJ → SBGO → SBCY → SBCG → SBCT → SBFL → SBPA → SBGR → SBGL → SBVT → SBCF → SBAR → SBSV → SBMO → SBJP → SBRF → SBSG → SBFZ → SBTE → SBSL → SBBE → SBMQ → SBBE → SBBV → SBEG → SBRB → SBPV → SBBR


| Métrica                        | Valor                  |
| ------------------------------ | ---------------------- |
| Total de códigos ICAO na lista | **29**                 |
| Capitais distintas             | **27**                             |
| Capitais repetidas             | SBBR (2 ×), SBBE (2 ×) |
|TOTAL EM HORAS                  |       **35 HORAS**


### **4. Visualização**

---
Nesta etapa, representamos graficamente a rota otimizada entre as 27 capitais, destacando a sequência de visitas e a direção dos voos, com base nas coordenadas geográficas de cada aeroporto.

(Impossivel visualizar com grafo normal)

---

## **4.1 – Obtenção das Coordenadas Geográficas**
1 - Carregamos um banco de dados global de aeroportos.

2 - Selecionamos apenas os 27 aeroportos das capitais brasileiras (baseado no código ICAO).

3 - Extraímos latitude e longitude.

4 - Salvamos essas coordenadas em um arquivo .csv para uso posterior na visualização.

[openflights](https://github.com/jpatokal/openflights)

In [240]:
cols = ["id","name","city","country","IATA","ICAO","lat","lon","alt","tz","DST","tz_db","type","source"]
airports = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat",
                       names=cols, index_col=False)

coords_df = airports[airports["ICAO"].isin(nodes)][["ICAO","lat","lon"]]
coords_df.to_csv("coords_27_capitais.csv", index=False)



## **4.2 – Visualização Interativa da Rota**
Cria-se um grafo com os voos da solução final e uma sequência ordenada de visita às capitais.

Em seguida:

1 - Convertemos os pontos em coordenadas (lat/lon).

2 - Geramos o mapa com as linhas do percurso e marcadores dos aeroportos.


3 - A visualização é feita com Plotly de forma interativa.




In [241]:
import pandas as pd, plotly.graph_objects as go, networkx as nx

# 1. Lê o arquivo com as coordenadas (latitude e longitude) dos 27 aeroportos das capitais
df_coords = pd.read_csv("coords_27_capitais.csv")  # O arquivo já tem as colunas ICAO, lat, lon
coords = {r.ICAO: (r.lon, r.lat) for r in df_coords.itertuples()}
# Criamos um dicionário → exemplo: {‘SBBR’: (-47.91, -15.86), ...}

# 2. Monta o grafo com os caminhos que foram escolhidos na solução do problema
start = "SBBR"  # Aeroporto inicial: Brasília
G = nx.MultiDiGraph()           # Grafo com múltiplas arestas possíveis entre pares de cidades
G.add_edges_from(tour_edges)   # Adiciona as arestas (voos) selecionadas na solução

# Verifica se o grafo tem um circuito euleriano (é necessário para percorrer tudo sem erro) #é um caminho que visita cada aresta de um grafo exatamente uma vez
assert nx.is_eulerian(G), "Algo deu errado: não dá para fazer um caminho completo passando por tudo!"

# 3. Encontra a sequência ideal de visita (a ordem das cidades a serem percorridas)
pairs = list(nx.eulerian_circuit(G, source=start))
rota = [start] + [v for (_, v) in pairs]  # Exemplo: [SBBR, SBGO, SBCG, ..., SBBR]

# 4. Monta listas com as posições (lon/lat) na ordem correta para desenhar o mapa
route_lon, route_lat = [], []
for a, b in zip(rota[:-1], rota[1:]):
    lon0, lat0 = coords[a]     # coordenadas do ponto de partida
    lon1, lat1 = coords[b]     # coordenadas do ponto de chegada
    route_lon += [lon0, lon1, None]  # adiciona os dois pontos e um "None" para quebrar a linha
    route_lat += [lat0, lat1, None]

# Lista de todos os pontos (aeroportos) para exibir no mapa
node_lon = [coords[n][0] for n in coords]
node_lat = [coords[n][1] for n in coords]

# 5. Cria a linha da rota no mapa
edge_trace = go.Scattergeo(
    lon=route_lon,
    lat=route_lat,
    mode="lines",                    # desenha como linhas
    line=dict(width=2, color="royalblue"),  # cor azul e espessura da linha
    opacity=0.7                      # transparência
)

# Cria os pontos (aeroportos) no mapa com seus nomes (códigos ICAO)
node_trace = go.Scattergeo(
    lon=node_lon,
    lat=node_lat,
    mode="markers+text",            # exibe ponto + nome
    text=list(coords.keys()),       # nomes dos aeroportos (SBBR, SBSP, ...)
    textposition="top center",      # posição do nome acima do ponto
    marker=dict(size=7, color="crimson")  # pontos vermelhos
)

# Monta o gráfico final com Plotly
fig = go.Figure(data=[edge_trace, node_trace])
fig.update_geos(
    projection_type="mercator",     # tipo de projeção do mapa
    fitbounds="locations",          # ajusta zoom automático
    showcountries=True,             # mostra fronteiras
    lataxis_range=[-35, 6],         # faixa de latitude para focar no Brasil
    lonaxis_range=[-75, -30]        # faixa de longitude para focar no Brasil
)
fig.update_layout(
    title="Rota Emergencial – Percurso Ótimo (interativo)",
    height=720,                     # altura do gráfico
    margin=dict(l=0, r=0, t=40, b=0) # margens
)
fig.show()  # Exibe o mapa interativo!

# Se quiser ver a ordem da rota como texto:
print("Sequência ordenada:")
print(" → ".join(rota))  # Ex: SBBR → SBGO → SBCG → ... → SBBR


Sequência ordenada:
SBBR → SBPJ → SBGO → SBCY → SBCG → SBCT → SBFL → SBPA → SBGR → SBGL → SBVT → SBCF → SBAR → SBSV → SBMO → SBJP → SBRF → SBSG → SBFZ → SBTE → SBSL → SBBE → SBMQ → SBBE → SBBV → SBEG → SBRB → SBPV → SBBR
